In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from keras_tuner import RandomSearch, BayesianOptimization
import keras
from keras.callbacks import EarlyStopping
import random

# Load the data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/BTC_FINAL_DF.csv')

# MAke the date the index
data.set_index('Date', inplace=True)


seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Normalize the data
# List of columns to scale
columns_to_scale = ['Open', 'High', 'Low', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
training_data = data[:training_size]
test_data = data[training_size:]

train_data = training_data[:int(len(training_data) * 0.9)]
val_data = training_data[int(len(training_data) * 0.9):]


# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)  
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3

# Target index ('Close')
close_index = 3

# Create the dataset
X_train, y_train = create_dataset(train_data, window_size, close_index)
X_test, y_test = create_dataset(test_data, window_size, close_index)
X_val, y_val = create_dataset(val_data, window_size, close_index)
x_train_full, y_train_full = create_dataset(training_data, window_size, close_index)

# Reshape the data
X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))
X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2], 1))
x_train_full = x_train_full.reshape((x_train_full.shape[0], window_size, x_train_full.shape[2], 1))

# Define a model-building function
def build_model(hp):
    model = Sequential()
    filters = hp.Int('filters', min_value=10, max_value=150, step=5)
    pool = hp.Choice('pool_size', values=[2, 3])
    model.add(Conv2D(filters=filters,
                    kernel_size=3, activation='relu', padding='same', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))
    
    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))

    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool,padding='same'))

    model.add(Flatten())

    model.add(Dense(1, activation='linear'))
    
    model.compile(optimizer=Adam(
        learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mae', 'mean_absolute_percentage_error'])
    
    return model

# Try 1: CNN-3D-REG
# Try 2: CNN-3D-REG-2
# Try 3: CNN-3D-REG-3
# Trt 4: CNN-3D-REG-Baye

tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_percentage_error',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-REG-4',
    overwrite=False
)

# Instantiate the tuner

# Configure EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mean_absolute_percentage_error',   
    patience=15,          
    restore_best_weights=True  
)

# Execute the search with EarlyStopping
tuner.search(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=2,
    callbacks=[early_stopping]  
)

# Get the best model
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)

print(best_model.summary())


Reloading Tuner from my_dir/CNN-3D-REG-4/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 3, 22, 50)      │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 2, 11, 50)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 2, 11, 120)     │        54,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 6, 120)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 6, 140)      │       151,340 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 3, 140)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 420)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           421 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 206,381 (806.18 KB)

 Trainable params: 206,381 (806.18 KB)

 Non-trainable params: 0 (0.00 B)

None


In [2]:
# Fit the model 30 times and get average metrics for test data
num_iterations = 30
mae_list = []
rmse_list = []
mape_list = []

for _ in range(num_iterations):
    best_hps = tuner.get_best_hyperparameters()[0]
    best_model = tuner.hypermodel.build(best_hps)

    history = best_model.fit(
        x_train_full, y_train_full,
        epochs=250,
        batch_size=32,
        verbose=0
    )

    predictions = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mape = tf.keras.losses.MAPE(y_test, predictions)

    mae_list.append(mae)
    rmse_list.append(rmse)
    mape_list.append(np.mean(mape))

# Calculate the average metrics
average_mae = np.mean(mae_list)
average_rmse = np.mean(rmse_list)
average_mape = np.mean(mape_list)

# Print the average metrics
print('Average Mean Absolute Error:', average_mae)
print('Average Root Mean Squared Error:', average_rmse)
print('Average Mean Absolute Percentage Error:', average_mape)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Average Mean Absolute Error: 955.1478448329348
Average Root Mean Squared Error: 1399.3961731819081
Average Mean Absolute Percentage Error: 2.3877504


In [3]:
# Seed
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Train the final model 
final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(
    x_train_full, y_train_full,
    epochs=250,
    batch_size=32,
    verbose=1
)

# Evaluate the final model on the test data
predictions = final_model.predict(X_test)
final_mae = mean_absolute_error(y_test, predictions)
final_rmse = np.sqrt(mean_squared_error(y_test, predictions))
final_mape = tf.keras.losses.MAPE(y_test, predictions)

print('Final Test Mean Absolute Error:', final_mae)
print('Final Test Root Mean Squared Error:', final_rmse)
print('Final Test Mean Absolute Percentage Error:', np.mean(final_mape))

# Save the final model
final_model.save('best_CNN3D.keras')

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 98809552.0000 - mae: 4420.3232 - mean_absolute_percentage_error: 29.9424 
Epoch 2/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1060383.3750 - mae: 515.6591 - mean_absolute_percentage_error: 3.4191
Epoch 3/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 1039365.6250 - mae: 511.3352 - mean_absolute_percentage_error: 3.3851
Epoch 4/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1018969.9375 - mae: 505.9202 - mean_absolute_percentage_error: 3.3946
Epoch 5/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 990490.8750 - mae: 497.1136 - mean_absolute_percentage_error: 3.3899
Epoch 6/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 922298.3125 - mae: 476.5972 - mean_absolute_percentage_error: 3.2499
Epoch 7/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 880663.1250 - mae: 465.8303 - mean_absolute_percentage_error: 3.1661
Epoch 8/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 861647.0625 - mae: 459.6744 -